### Michael Beven - 455613
### University of Chicago - Financial Mathematics
### FINM 33150 - Quantitative Strategies and Regression
### Homework 3

#### Tables and column descriptions

* **The strategy dataframe**: 
 * ``` XP ``` ~ Adjusted price of security X
 * ``` XV ``` ~ Adjusted volume of security X
 * ``` YP ``` ~ Adjusted price of security Y
 * ``` YV ``` ~ Adjusted volume of security Y
 * ``` XDDV ``` ~ Daily dollar volume of security X
 * ``` Nt ``` ~ 15 day rolling median of ``` XDDV ```
 * ``` XR ``` ~ Log return of X
 * ``` YR ``` ~ Log return of Y
 * ``` Delta ``` ~ ``` XR ``` - ``` YR ```
 * ``` DeltaM ``` ~ M day sume of ``` Delta ```
 * ``` Signal ``` ~ Buy, short, exit signal (-1, 0 or 1)
 * ``` EOM ``` ~ End of month indicator
 * ``` Entry ``` ~ Entry day indicator
 * ``` Exit ``` ~ Exit day indicator
 * ``` Nx ``` ~ Number of X shares bought/shorted
 * ``` Ny ``` ~ Number of Y shares bought/shorted
 * ``` Profit ``` ~ ``` Signal ```(``` Nx ``` $\times$ ``` XR ``` - ``` Ny ``` $\times$ ``` YR```)
 * ``` Cum_Profit ``` ~ Cumulative profit
 * ``` K ``` ~ Cumulative capital
 * ``` Return  ``` ~ Annualised ``` Profit ``` / ``` K ```
* **ETF simulation arguments** (``` sims ```): 
 * ``` M ``` ~ Number of days for the M day return
 * ``` g ``` ~ Entry barrier
 * ``` j ``` ~ Exit barrier
 * ``` X_code ``` ~ Quandl code of security X
 * ``` Y_code ``` ~ Quandl code of security Y
 * ``` X_close ``` ~ Column name for adjusted close price of X
 * ``` X_volume ``` ~ Column name for adjusted volume of X
 * ``` Y_close ``` ~ Column name for adjusted close price of Y
 * ``` Y_colume ``` ~ Column name for adjusted volume of Y
* **Fama-French daily factor returns** (``` FF_data ```): 
 * ``` Mkt-RF ``` ~ Market return
 * ``` SMB ``` ~ Small minus big return
 * ``` HML ``` ~ High minus low return
 * ``` RF ``` ~ Risk free return
* **Sharpe and Sortino ratios on returns** (``` Ratios ```): 
 * ``` ETF ``` ~ ETF number 
 * ``` Sharpe ``` ~ Sharpe ratio: $\frac{E[r-r_b]}{\sqrt{(r-r_b)^2}}$
 * ``` Sortino ``` ~ Sortino ratio: $\frac{E[r-r_b]}{\sqrt{(r-r_b)^2 | r > r_b}}$
* **Individual regressions** (``` Indiv ```): 
 * ``` ETF ``` ~ ETF number
 * ``` SMB ``` ~ SMB regression coefficient
 * ``` SMB Sharpe ``` ~ SMB regression Sharpe ratio
 * ``` SMB Sortino ``` ~ SMB regression Sortino ratio
 * ``` HML ``` ~ HML regression coefficient
 * ``` HML Sharpe ``` ~ HML regression Sharpe ratio
 * ``` HML Sortino ``` ~ HML regression Sortino ratio
 * ``` RF ``` ~ RF regression coefficient
 * ``` RF Sharpe ``` ~ RF regression Sharpe ratio
 * ``` RF Sortino ``` ~ RF regression Sortino ratio
 * ``` Mkt-RF ``` ~ Mkt-RF regression coefficient
 * ``` Mkt-RF Sharpe ``` ~ Mkt-RF regression Sharpe ratio
 * ``` Mkt-RF Sortino ``` ~ Mkt-RF regression Sortino ratio
* **Multivariate regressions** (``` Multi ```): 
 * ``` ETF ``` ~ ETF number 
 * ``` SMB ``` ~ SMB regression coefficient
 * ``` HML ``` ~ HML regression coefficient
 * ``` RF ``` ~ RF regression coefficient
 * ``` Mkt-RF ``` ~ Mkt-RF regression coefficient
 * ``` Sharpe ``` ~ Regression Sharpe ratio
 * ``` Sortino ``` ~ Regression Sortino ratio

In [5]:
# Michael Beven - 455613
# University of Chicago - Financial Mathematics
# FINM 33150 - Quantitative Strategies and Regression
# Homework 3 

# make plots come up in this window - ipython notebook
%matplotlib inline

# import packages
import pandas as pd
import numpy as np
import Quandl
from statsmodels.regression.linear_model import OLS

# set global variables
auth = "v21snmSix9KyXBWc1RkF"
start_date = "2013-12-02"
trade_begin = "2014-01-01"
end_date = "2015-12-31"

In [6]:
# reversion strategy function
def strat(M,g,j,X_code,Y_code,X_close,X_volume,Y_close,Y_volume):
  """
  This function creates a dataframe with results to a spread trading strategy
  (see HW2 of FINM 33150 - Quantitative Strategies and Regression)
  Inputs:
  M ~ return difference calculation time frame.  M cannot exceed the number of 
  trading days between 2013-12-02 and 2014-01-01
  g ~ entering threshold
  j ~ exiting threshold
  s ~ stop loss threshold
  X_code ~ Quandl code for X
  Y_code ~ Quandl code for Y
  X_close ~ X column name for close
  X_volume ~ X column name for volume
  Y_close ~ Y column name for close
  Y_volume ~ Y column name for volume
  Example of calling function:
  strat(10,0.01,0.008,0.10,'GOOG/NYSE_XSD','YAHOO/SMH','GOOG.NYSE_XSD - Close',
  'GOOG.NYSE_XSD - Volume','YAHOO.SMH - Close','YAHOO.SMH - Volume')
  """
  # grab data using Quandl
  ETF_data = Quandl.get(list((X_code,Y_code)),authtoken=auth,trim_start=start_date,trim_end=end_date,returns="pandas")
  df = pd.DataFrame(ETF_data.ix[:,(X_close,X_volume,Y_close,Y_volume)]) #subset
  df.columns = ['XP','XV','YP','YV']
  df['XDDV'] = df.XP*df.XV # calculate daily dollar volumes
  df['Nt'] = pd.rolling_median(df.XDDV,15)# 15 day rolling median
  K = np.max(2*df.Nt) # capital - set K now that we have Nt
  df['XR'] = np.log(df.XP) - np.log(df.XP.shift(1)) #logrets
  df['YR'] = np.log(df.YP) - np.log(df.YP.shift(1))
  df['Delta'] = df.XR-df.YR # difference of X and Y
  df['DeltaM'] = pd.rolling_sum(df.Delta,M)#M day accumulated difference 
  df = df[df.index >= trade_begin] # drop unnecessary date range
  df['Signal'] = np.nan # add empty trade signal column
  df.Signal[df.DeltaM > g] = 1 # entering or maintaining trade
  df.Signal[df.DeltaM < -g] = -1 # entering or maintaining trade
  df.Signal[np.abs(df.DeltaM) < j] = 0 # exiting or out of trade
  df['EOM'] = np.nan # end of month
  df.EOM[(df.shift(1,freq='B').index.day <= 3) & (df.shift(1,freq='B').index.day-df.index.day < -1)] = 1 # day before 1st day
  df.Signal[(df.shift(1,freq='B').index.day <= 3) & (df.shift(1,freq='B').index.day-df.index.day < -1)] = 0
  for i in range(1,len(df)):
    if np.isnan(df.Signal[i]):# if between g and j
      df.Signal[i] = df.Signal[i-1] # fill in with current position
  df.Signal[((df.Signal == -1) & (df.DeltaM > j)) | (df.Signal == 1) & (df.DeltaM < j)] = 0
  df['Entry'] = 1*(((df.Signal == 1) | (df.Signal == -1)) & (df.shift(1).Signal == 0)) # entry point
  df['Exit'] = 1*((df.Signal == 0) & ((df.shift(1).Signal == 1) | (df.shift(1).Signal == -1))) # exit point
  df['Nx'] = np.round(df.Signal*df.Nt/100/df.XP,0) # size of X trade
  df['Ny'] = np.round(df.Signal*df.Nt/100/df.YP,0) # size of Y trade
  df['Profit'] = pd.DataFrame(df.Signal*(df.Nx.shift(1)*df.XP.shift(1)*df.XR)-df.Ny.shift(1)*df.YP.shift(1)*df.YR) # dollar profit(loss)
  df['Cum_Profit'] = np.cumsum(df.Profit) #cumulative profit
  df['K'] = np.round(K + df.Cum_Profit,0) # capital based on changes in profit  
  df['Return'] = 252*df.Profit/df.K.shift(1) # annualised returns  
  df = np.round(df,3) # round
  return df

In [7]:
# set up a dataframe of the simulations to run
sims = pd.DataFrame(columns=['M','g','j','X_code','Y_code','X_close','X_volume','Y_close','Y_volume'])
sims.loc[0] = [20,0.02,0.001,'EOD/RYU','EOD/XLU','EOD.RYU - Adj_Close','EOD.RYU - Adj_Volume','EOD.XLU - Adj_Close','EOD.XLU - Adj_Volume']
sims.loc[1] = [20,0.060,0.047,'EOD/IST','EOD/IYZ','EOD.IST - Adj_Close','EOD.IST - Adj_Volume','EOD.IYZ - Adj_Close','EOD.IYZ - Adj_Volume']
sims.loc[2] = [20,0.021,0.003,'EOD/RING','EOD/GDX','EOD.RING - Adj_Close','EOD.RING - Adj_Volume','EOD.GDX - Adj_Close','EOD.GDX - Adj_Volume']
sims.loc[3] = [20,0.045,0.003,'EOD/XSD','EOD/SMH','EOD.XSD - Adj_Close','EOD.XSD - Adj_Volume','EOD.SMH - Adj_Close','EOD.SMH - Adj_Volume']
sims.loc[4] = [20,0.0025,0.0015,'EOD/PBE','EOD/XBI','EOD.PBE - Adj_Close','EOD.PBE - Adj_Volume','EOD.XBI - Adj_Close','EOD.XBI - Adj_Volume']
sims.loc[5] = [20,0.02,0.014,'EOD/PXJ','EOD/OIH','EOD.PXJ - Adj_Close','EOD.PXJ - Adj_Volume','EOD.OIH - Adj_Close','EOD.OIH - Adj_Volume']
sims.loc[6] = [20,0.034,0.0015,'EOD/IEO','EOD/XOP','EOD.IEO - Adj_Close','EOD.IEO - Adj_Volume','EOD.XOP - Adj_Close','EOD.XOP - Adj_Volume']
sims.loc[7] = [20,0.055,0.013,'EOD/RTH','EOD/XRT','EOD.RTH - Adj_Close','EOD.RTH - Adj_Volume','EOD.XRT - Adj_Close','EOD.XRT - Adj_Volume']
sims.loc[8] = [20,0.0019,0.0007,'EOD/SIVR','EOD/SLV','EOD.SIVR - Adj_Close','EOD.SIVR - Adj_Volume','EOD.SLV - Adj_Close','EOD.SLV - Adj_Volume']
sims.loc[9] = [20,0.08,0.01,'EOD/HYLD','EOD/JNK','EOD.HYLD - Adj_Close','EOD.HYLD - Adj_Volume','EOD.JNK - Adj_Close','EOD.JNK - Adj_Volume']
print(sims)

    M       g       j    X_code   Y_code               X_close  \
0  20  0.0200  0.0010   EOD/RYU  EOD/XLU   EOD.RYU - Adj_Close   
1  20  0.0600  0.0470   EOD/IST  EOD/IYZ   EOD.IST - Adj_Close   
2  20  0.0210  0.0030  EOD/RING  EOD/GDX  EOD.RING - Adj_Close   
3  20  0.0450  0.0030   EOD/XSD  EOD/SMH   EOD.XSD - Adj_Close   
4  20  0.0025  0.0015   EOD/PBE  EOD/XBI   EOD.PBE - Adj_Close   
5  20  0.0200  0.0140   EOD/PXJ  EOD/OIH   EOD.PXJ - Adj_Close   
6  20  0.0340  0.0015   EOD/IEO  EOD/XOP   EOD.IEO - Adj_Close   
7  20  0.0550  0.0130   EOD/RTH  EOD/XRT   EOD.RTH - Adj_Close   
8  20  0.0019  0.0007  EOD/SIVR  EOD/SLV  EOD.SIVR - Adj_Close   
9  20  0.0800  0.0100  EOD/HYLD  EOD/JNK  EOD.HYLD - Adj_Close   

                X_volume              Y_close              Y_volume  
0   EOD.RYU - Adj_Volume  EOD.XLU - Adj_Close  EOD.XLU - Adj_Volume  
1   EOD.IST - Adj_Volume  EOD.IYZ - Adj_Close  EOD.IYZ - Adj_Volume  
2  EOD.RING - Adj_Volume  EOD.GDX - Adj_Close  EOD.GDX - Adj_Vo

In [8]:
# create all data frames
dfs = {}
for i in range(0,9):
  dfs['df'+str(i)] = strat(sims.ix[i,'M'],sims.ix[i,'g'],sims.ix[i,'j'],sims.ix[i,'X_code'],sims.ix[i,'Y_code'],sims.ix[i,'X_close'],sims.ix[i,'X_volume'],sims.ix[i,'Y_close'],sims.ix[i,'Y_volume'])

In [9]:
# FF data
FF_data = Quandl.get('KFRENCH/FACTORS_D',authtoken=auth,trim_start=trade_begin,trim_end=end_date,returns="pandas")
print(FF_data.head())

            Mkt-RF   SMB   HML  RF
Date                              
2014-01-02   -0.88 -0.24  0.11   0
2014-01-03    0.03  0.37  0.03   0
2014-01-06   -0.34 -0.56  0.26   0
2014-01-07    0.68  0.37 -0.39   0
2014-01-08    0.04  0.02 -0.23   0


In [10]:
# ratios
Ratios = pd.DataFrame(columns=['ETF','Sharpe','Sortino'])  
for i in range(0,9):
  df = dfs['df'+str(i)]
  Sharpe = (np.mean(df.Return - FF_data['RF']/100))/np.sqrt(np.mean(np.power(df.Return - FF_data['RF']/100,2)))
  Sortino = (np.mean(df.Return - FF_data['RF']/100))/np.sqrt(np.mean(np.power(df.Return[df.Return > FF_data['RF']] - FF_data['RF'][df.Return > FF_data['RF']],2)))
  Ratios.loc[i] = ['df'+str(i),Sharpe,Sortino]
print(Ratios)

   ETF    Sharpe   Sortino
0  df0  0.012733  0.004627
1  df1  0.037462  0.009688
2  df2  0.050536  0.016579
3  df3  0.012591  0.006828
4  df4  0.035155  0.034704
5  df5  0.033817  0.013325
6  df6  0.021711  0.011802
7  df7  0.040183  0.011838
8  df8  0.039080  0.013594


In [11]:
# individual regressions
Indiv = pd.DataFrame(columns=['ETF','SMB','SMB Sharpe','SMB Sortino','HML','HML Sharpe','HML Sortino','RF','RF Sharpe','RF Sortino','Mkt-RF','Mkt-RF Sharpe','Mkt-RF Sortino'])
for i in range(0,9):
  df = dfs['df'+str(i)]
  SMB = OLS(df.Return,FF_data.SMB,missing='drop').fit()
  SMB_p = SMB.params['SMB']
  SMB_Sharpe = np.mean(SMB.resid)/np.sqrt(np.mean(np.power(SMB.resid,2)))
  SMB_Sortino = np.mean(SMB.resid)/np.sqrt(np.mean(np.power(SMB.resid[SMB.resid>0],2)))
  HML = OLS(df.Return,FF_data.HML,missing='drop').fit()
  HML_p = HML.params['HML']
  HML_Sharpe = np.mean(HML.resid)/np.sqrt(np.mean(np.power(HML.resid,2)))
  HML_Sortino = np.mean(HML.resid)/np.sqrt(np.mean(np.power(HML.resid[HML.resid>0],2)))
  RF = OLS(df.Return,FF_data.RF,missing='drop').fit()
  RF_p = RF.params['RF']
  RF_Sharpe = np.mean(RF.resid)/np.sqrt(np.mean(np.power(RF.resid,2)))
  RF_Sortino = np.mean(RF.resid)/np.sqrt(np.mean(np.power(RF.resid[RF.resid>0],2)))
  MktRF = OLS(df.Return,FF_data['Mkt-RF'],missing='drop').fit()
  MktRF_p = MktRF.params['Mkt-RF']
  MktRF_Sharpe = np.mean(MktRF.resid)/np.sqrt(np.mean(np.power(MktRF.resid,2)))
  MktRF_Sortino = np.mean(MktRF.resid)/np.sqrt(np.mean(np.power(MktRF.resid[MktRF.resid>0],2)))
  Indiv.loc[i] = ['df'+str(i),SMB_p,SMB_Sharpe,SMB_Sortino,HML_p,HML_Sharpe,HML_Sortino,RF_p,RF_Sharpe,RF_Sortino,MktRF_p,MktRF_Sharpe,MktRF_Sortino]
print(Indiv)

   ETF       SMB  SMB Sharpe  SMB Sortino       HML  HML Sharpe  HML Sortino  \
0  df0  0.000174    0.016196     0.018057  0.000056    0.014043     0.016142   
1  df1 -0.000424    0.033214     0.027961 -0.000086    0.036269     0.029166   
2  df2  0.001373    0.052621     0.045841  0.003268    0.057264     0.051435   
3  df3  0.001134    0.015514     0.019016 -0.001376    0.007901     0.008924   
4  df4  0.004707    0.045477     0.048158 -0.007065    0.017492     0.018163   
5  df5  0.000957    0.035989     0.031338  0.002273    0.040792     0.036584   
6  df6 -0.001655    0.015290     0.013502 -0.001541    0.014725     0.012850   
7  df7 -0.000305    0.035316     0.030708 -0.000111    0.037719     0.031358   
8  df8 -0.000147    0.038238     0.034430  0.000207    0.040809     0.039392   

   RF  RF Sharpe  RF Sortino    Mkt-RF  Mkt-RF Sharpe  Mkt-RF Sortino  
0   0   0.012733    0.004627  0.000129       0.008334        0.008787  
1   0   0.037462    0.009688  0.000461       0.031899  

In [12]:
# mutlvariate regressions
Multi = pd.DataFrame(columns=['ETF','SMB','HML','RF','Mkt-RF','Sharpe','Sortino'])
for i in range(0,9):
  df = dfs['df'+str(i)]
  Reg = OLS(df.Return,FF_data,missing='drop').fit()
  Reg_p = Reg.params
  Reg_Sharpe = np.mean(Reg.resid)/np.sqrt(np.mean(np.power(Reg.resid,2)))
  Reg_Sortino = np.mean(Reg.resid)/np.sqrt(np.mean(np.power(Reg.resid[Reg.resid>0],2)))
  Multi.loc[i] = ['df'+str(i),Reg_p['SMB'],Reg_p['HML'],Reg_p['RF'],Reg_p['Mkt-RF'],Reg_Sharpe,Reg_Sortino]
print(Multi)  


   ETF       SMB       HML  RF    Mkt-RF    Sharpe   Sortino
0  df0  0.000202  0.000167   0  0.000134  0.016124  0.017942
1  df1 -0.000541 -0.000129   0  0.000482  0.024219  0.020040
2  df2  0.002556  0.004215   0  0.000088  0.062989  0.056179
3  df3  0.000647 -0.000480   0  0.002126  0.005899  0.006980
4  df4  0.002759 -0.005173   0  0.002857  0.021635  0.022539
5  df5  0.001524  0.003766   0  0.002998  0.041745  0.036187
6  df6 -0.002271 -0.002524   0 -0.000558  0.003738  0.003190
7  df7 -0.000347 -0.000336   0 -0.000318  0.034803  0.031298
8  df8 -0.000092  0.000151   0 -0.000075  0.040420  0.038836
